In [2]:
import numpy as np
import pyedflib
import dask
from dask.delayed import delayed
import dask.array as da
import os
import pickle
import sys
sys.path.insert(0,'/scratch/mcesped/code/NoiseDetection_iEEG/interictal_classifier/')
from features import get_DWT_features 

### Functions

In [28]:
import dask.array as da

@dask.delayed
def load_segment(file_path, channel_name, n_samples):
    with pyedflib.EdfReader(file_path) as f:
        edf_chns = f.getSignalLabels()
        # Load the specified segment from the file
        segment_data = f.readSignal(edf_chns.index(channel_name), n=n_samples)
    return segment_data


# Assuming `segments_array` is a Dask array with the shape (n_channels, n_segments, samples_per_epoch)
# and `rf_model` is your pre-trained RandomForestClassifier
def predict(data):
    shape = len(data)
    random_array = np.random.choice([0, 1, 2], size=shape)
    return random_array

def get_features(data): 
    return np.vstack([np.mean(data, axis=1), np.median(data, axis=1)]).T

def predict_channel_epochs(channel_data, model):
    # Compute the Dask array for the current channel to a NumPy array
    channel_data_np = np.array(np.split(channel_data.compute(), len(channel_data)/channel_data.chunks[0][0])) 
    # Predict
    features = []
    for epoch in channel_data_np:
        features.append(get_DWT_features(epoch, "db4"))
    features = np.array(features)
    predictions = model.predict_proba(features)
    return predictions

### Compute

In [16]:
# Load model
filename = 'RF_model.sav'
model_rf = pickle.load(open(filename, 'rb'))
# model_rf.set_params(n_jobs = 8)
model_rf

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=3, max_features=0.5, min_samples_leaf=0.05,
                       n_estimators=250, n_jobs=32)

In [53]:
# Files and channels
files_and_channels = [
    ('/home/mcesped/scratch/Results/seegprep/ml/work/sub-103/ses-003/ieeg/sub-103_ses-003_task-full_rec-PLIreject_run-01_clip-01_ieeg.edf',
    ["RAHc1-2","RAHc2-3","RAHc3-4","RAHc4-5","RAIn1-2","RAIn2-3","RAIn3-4","RAIn5-6","LAHc1-2","LAHc2-3","LAHc3-4","LAHc4-5","LOFr3-4","LOFr4-5","LOFr5-6","LOFr6-7"]),
    ('/home/mcesped/scratch/Results/seegprep/ml/work/sub-087/ses-007/ieeg/sub-087_ses-007_task-full_rec-PLIreject_run-01_clip-01_ieeg.edf',
    ["LAm1-2","LAm2-3","LAm3-4","LAm8-9","RAIn6-7","RAIn7-8","RAIn8-9","RAIn3-4","ROFr1-2","ROFr4-5","ROFr7-8","ROFr8-9","LOFr4-5","LOFr5-6","LOFr6-7","LOFr7-8"]),
    ('/home/mcesped/scratch/Results/seegprep/ml/work/sub-093/ses-004/ieeg/sub-093_ses-004_task-full_rec-PLIreject_run-01_clip-01_ieeg.edf',
    ["LITeLs2-3","LITeLs3-4","LITeLs6-7","LITeLs8-9","LPOFr1-2","LPOFr2-3","LPOFr4-5","LPOFr8-9","LAOFr1-2","LAOFr2-3","LAOFr3-4","LAOFr4-5","LOpS1-3-4","LOpS1-5-6","LOpS1-6-7","LOpS1-7-8"]),
    ('/home/mcesped/scratch/Results/seegprep/ml/work/sub-081/ses-006/ieeg/sub-081_ses-006_task-full_rec-PLIreject_run-01_clip-01_ieeg.edf',
    ["LMHc1-2","LMHc2-3","LMHc3-4","LMHc4-5","RAm1-2","RAm2-3","RAm3-4","RAm4-5","RAIn1-2","RAIn3-4","RAIn4-5","RAIn5-6","LPNeC5-6","LPNeC6-7","LPNeC7-8","LPNeC8-9"]),
]
# files_and_channels = [
#     ('/home/mcesped/scratch/Results/seegprep/ml/bids/sub-103/ses-003/ieeg/sub-103_ses-003_task-full_rec-regionID_run-01_clip-01_ieeg.edf',
#     ["RAHc1-2"])
# ]
out_dir = '/home/mcesped/scratch/Results/ML_continous_data'

In [55]:
n = 0
for file_path, channels in files_and_channels:
    n+=len(channels)
n

64

In [54]:
# Process and classify epochs for each file, using Dask for epoch storage
for file_path, channels in files_and_channels:
    # Get number of chunks
    with pyedflib.EdfReader(file_path) as edf:
        n_samples = edf.getNSamples()[0]
        sample_rate = edf.getSampleFrequencies()[0]# / edf.datarecord_duration #UNCOMENT FOR pyedlib < 0.1.32
    samples_per_epoch = int(3 * sample_rate)  # 3 seconds of data
    chunks = n_samples // samples_per_epoch
    segment_tasks = [load_segment(file_path, ch, chunks*samples_per_epoch) for ch in channels]
    segments_array = da.stack([da.from_delayed(task, shape=(chunks*samples_per_epoch,), dtype=float).rechunk((samples_per_epoch,)) for task in segment_tasks])
    # Predict per channel
    for i in range(segments_array.shape[0]):  # Loop over channels
        channel_data = segments_array[i, :]
        predictions_prob = predict_channel_epochs(channel_data, model_rf)
        # Save pred prob
        os.makedirs(os.path.join(out_dir, 'prob'), exist_ok=True)
        out_file = os.path.join(out_dir, 'prob', os.path.basename(file_path).replace('ieeg.edf', f"chn-{channels[i]}_pred.npy"))
        np.save(out_file, predictions_prob)
        # Produce an output
        os.makedirs(os.path.join(out_dir, 'edf'), exist_ok=True)
        out_file = os.path.join(out_dir, 'edf', os.path.basename(file_path).replace('ieeg.edf', f"chn-{channels[i]}_ieeg.edf"))
        write_ML_results(file_path, out_file, predictions_prob, epoch_duration_seconds=3)

# Create hypnogram

In [51]:
def write_ML_results(original_file, new_file, predictions_prob, epoch_duration_seconds):
    from pyedflib import EdfReader, EdfWriter
    import numpy as np

    # Step 1: Read the original EDF file
    with EdfReader(original_file) as edf:
        # Gather signal and header information
        n_signals = edf.signals_in_file
        signal_headers = edf.getSignalHeaders()
        header = edf.getHeader()
        #For pyedlib < 0.1.32
        datarecord_duration = int(edf.datarecord_duration * 100000)
        # else
        # datarecord_duration = int(edf.datarecord_duration)

        # Read signal data
        signals = [edf.readSignal(i) for i in range(n_signals)]
        

    # Step 2: Create a new EDF file, preserving header and signal information but excluding annotations
    with EdfWriter(new_file, n_signals, file_type=pyedflib.FILETYPE_EDFPLUS) as out_edf:
        # Write header information
        out_edf.setHeader(header)
        out_edf.setSignalHeaders(signal_headers)
        # out_edf.setDatarecordDuration(datarecord_duration)

        # Write signal data
        out_edf.writeSamples(signals)

        
        # Convert predictions to string
        # First get labels
        predictions = predictions_prob.argmax(axis=1)
        # labels = np.array(['Noise', 'Pathology', 'Baseline'])
        # mapped_labels = labels[predictions]
        # # Now the probabilities
        # probabilities = predictions_prob.max(axis=1)
        # final_array = np.array([f'{label} {prob*100:.2f}%' for label, prob in zip(mapped_labels, probabilities)])
        
        # Step 3: Add new annotations
        new_annotations = [(i*epoch_duration_seconds, epoch_duration_seconds, f'{cat}') for i, cat in enumerate(predictions)]
        for onset, duration, description in new_annotations:
            out_edf.writeAnnotation(onset, duration, description)

In [ ]:
a = 

In [5]:
from pyedflib import EdfWriter
import pyedflib
import numpy as np

# Define the filename
filename = 'sleep_data.edf'

# Create a new EDF file
with EdfWriter(filename, n_channels=1, file_type=pyedflib.FILETYPE_EDFPLUS) as f:
    # Set signal headers (adjust parameters as needed)
    signal_headers = [{'label': 'Dummy Signal', 'dimension': 'uV', 'sample_rate': 1, 'physical_max': 100, 'physical_min': -100, 'digital_max': 32767, 'digital_min': -32768}]
    f.setSignalHeaders(signal_headers)
    
    # Generate dummy signal data (replace with your actual data)
    data = np.zeros(100)  # 100 seconds of dummy data
    f.writeSamples([data])
    
    # Add sleep stage annotations
    # The annotations are tuples in the form (onset, duration, annotation)
    # Onset is in seconds from the start of the recording
    annotations = [
        (0, 30, 'Baseline'),
        (30, 30, 'Pathology'),
        (60, 30, 'Noise'),
        (90, 30, 'Pathology'),
        # Add more stages as needed
    ]
    for onset, duration, stage in annotations:
        f.writeAnnotation(onset, duration, stage)

# The EDF file is now created with the dummy signal and annotations